In [1]:
from sage.all import *
import random
from sage.crypto.sboxes import sboxes
from algorithm import Algorithm
from greedy_extensions import Greedy
S = sboxes["SKINNY_8"]
A = Greedy(S, n=S.input_size(), m=S.output_size())
n = A.n
m = A.m
Si = [list(S).index(x) for x in range(2**n)]
approx = []
for _ in tqdm(range(2000)):
    #xs = A.sample_kzerosum(4)
    xs = [randrange(2**A.n)]
    for sol, profile in A.run_from(xs, limit=1):
        if sol not in approx and len(sol) > 40:
            approx.append(sol)
            #print(len(sol), hex(hash(sol)), "profile", profile, len(set(sols)), "/", len(sols))
len(approx), Counter(map(len, approx))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:10<00:00, 193.69it/s]


(485, Counter({42: 149, 44: 87, 41: 80, 45: 76, 43: 49, 47: 40, 46: 4}))

In [2]:
from ortools.linear_solver import pywraplp

#BOUND = 2.0**-n / 10.0
BOUND = 2**-8.0
print("BOUND", BOUND, "=", BOUND * 2**n, "/2^n")

sets = approx[:]
k = len(sets)

solver = pywraplp.Solver.CreateSolver('SCIP')

wt = [
    solver.Var(0, 1, false, "wt%d" % 0)
    for i in range(k)
]
solver.Add(sum(wt) == 1.0)
           
vec = [[] for _ in range(2**n)]
for i, St in enumerate(sets):
    for x in St:
        vec[x].append(wt[i])

vec = [solver.Sum(x) for x in vec]

# faster but less precise (compare to min prob.)
#min_i = randrange(2**n) # guess the minimum element
#inds = [i for i in range(2**n) if i != min_i]
#print("min_i", min_i)
# for i in inds:
#     solver.Add(vec[i] - vec[min_i] >= 0)
#     solver.Add(vec[i] - vec[min_i] <= BOUND)

# slower but more precise (compare to avg prob.)
# slow because of the interface...
avg = solver.Sum(vec) / len(vec)
for i in range(2**n):
    solver.Add(vec[i] - avg >= -BOUND)
    solver.Add(vec[i] - avg <= BOUND)

solver.Maximize(sum(len(S) * wt[i] for i, S in enumerate(sets)))

solver.SetTimeLimit(5_000) # in milliseconds

# 0: optimal solution
# 1: solution found (suboptimal?)
# else: not ready yet
status = solver.Solve()
print("status", status)
if status == 2:
    print("NO SOLUTION! Need more approximations ot reduce the bound.")
if status in (0, 1):
    print("objective", solver.Objective().Value(), "= expected approx. size")
    sol = [wt[i].solution_value() for i in range(k)]
    probs = [v.solution_value() for v in vec]
    print("min prob", min(probs), "max prob", max(probs))
    print("avg prob", sum(probs) / len(probs))
    dev = (max(probs) - min(probs)) / 2
    print("max deviation", dev, "= 1 /", 1/dev)
    # for i in range(k):
    #     if sol[i] > 1e-9:
    #         print(i, ":", sol[i])
    # print(sol)
    # for v in vector:
    #     print("v", v.solution_value())
    weights = sol

<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type swigvarlink has no __module__ attribute


BOUND 0.00390625 = 1.0 /2^n
status 0
objective 43.70223513151977 = expected approx. size
min prob 0.16680560598249383 max prob 0.17461810598250346
avg prob 0.1707118559824991
max deviation 0.003906250000004816 = 1 / 255.9999999996844


In [3]:
from binteger import Bin
mats = []
for s in tqdm(sets):
    assert A.is_affine_approximation(s)
    aff = A.approximation_from_inputs(s)
    
    const = Bin(aff[0], n).vector
    lin = [y ^ aff[0] for y in aff]
    
    mat = matrix(GF(2), [
        Bin(lin[2**(n-1-i)], n) for i in range(n)
    ]).transpose()

    for x in Bin.iter(n):
        y = mat * vector(x.vector) + const
        assert aff[x.int] == Bin(y).int, x
    mats.append((mat, const))
        
xs, = random.choices(sets, weights=weights)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 485/485 [00:05<00:00, 92.00it/s]


Note: due to $c=r$, not all random instances have a solution. If a solution is not found, regenerate the instance.

In [43]:
c = r = 2 # 16 + 16 bits
c = r = 3 # 24 + 24 bits
m = c + r
NR = 3
LIN = GL(m*n, GF(2)).random_element().matrix()
c, r, LIN

(3, 3, 48 x 48 dense matrix over Finite Field of size 2)

In [44]:
def P(xs):
    state = list(xs)
    for i in range(NR):
        state = [S[x] for x in state]
        #print("S")
        if i < NR - 1:
            #print("L")
            state = Bin.concat(*state, n=n).vector
            state = LIN * state
            state = [v.int for v in Bin(state).split(n=n)]
    #print()
    return state

P([0] * (c+r)), P([1] + [0] * (c+r-1))

([254, 83, 80, 44, 38, 232], [197, 81, 35, 5, 71, 64])

In [ ]:
from itertools import product
lst = product(range(2**n), repeat=c)
for x in tqdm(lst):
    x = [0] * c + list(x)
    y = P(x)
    if y[:c] == [0]*c:
        print("good", x)

In [ ]:
tries = 0
wins = 0

In [51]:
import time
t_start = time.time()
while True:
    tries += 1
    if tries % 1000 == 0:
        print(tries, ct, apps[1], "per iter", (time.time() - t_start) / tries)
    
    apps = []
    inds = list(range(len(sets)))
    for i in range(NR):
        row = []
        for j in range(m):
            app_i, = random.choices(inds, weights=weights)
            row.append(app_i)
        apps.append(row)
        #print(row)
    
    #xs = BooleanPolynomialRing(n*c, names='x').gens()
    xs = PolynomialRing(GF(2), names=["x%d" % i for i in range(n*c)]).gens()
    
    # S layer: free
    state = [Bin(S[0], n).vector] * c + [xs[i:i+n] for i in range(0, len(xs), n)]

    # L layer
    state = sum(map(list, state), [])
    state = LIN * vector(state)
    state = [state[i:i+n] for i in range(0, len(state), n)]

    # S layer (middle)
    for i in range(len(state)):
        app_i = apps[1][i]  # middle round approx.
        mat, const = mats[app_i]
        state[i] = mat * state[i] + const

    # L layer
    state = sum(map(list, state), [])
    state = LIN * vector(state)
    #state = [state[i:i+n] for i in range(0, len(state), n)]

    # S layer: free
    output = state[:c*n]
    target = vector(GF(2), Bin(Si[0], n).list * c)
    seq = Sequence(output - target)
    
    mat, monos = seq.coefficients_monomials()
    if mat.ncols() != c*n+1:
        continue
    #print("ok")
    tar = mat.column(-1)
    mat = mat[:,:-1]
    try:
        base_sol = mat.solve_right(tar)
    except:
        continue
    for zero in mat.right_kernel():
        sol = base_sol + zero
        #print("sol", sol)
        #sol = sol[::-1]
        pt = [0] * c + [Si[v.int] for v in Bin(sol).split(r)]
        #print(pt, P(pt))
        ct = P(pt)
        if ct[:c] == [0]*c:
            wins += 1
            print("good", pt, ct, ":", wins / tries, math.log(wins / tries, 2))

280000 [175, 231, 2, 96, 167, 62] [475, 30, 244, 256, 78, 3] per iter 1.7222248656409128e-05
281000 [95, 67, 58, 76, 85, 29] [460, 108, 181, 400, 343, 373] per iter 5.998265361446503e-05
good [0, 0, 0, 192, 7, 211] [0, 0, 0, 108, 245, 26] : 5.6814751950343905e-05 -14.103374900299174
282000 [94, 121, 35, 229, 248, 102] [365, 76, 145, 30, 183, 384] per iter 0.00011075867744202309
283000 [211, 128, 202, 175, 235, 59] [98, 478, 36, 107, 381, 345] per iter 0.0001624340724608081
284000 [126, 185, 136, 164, 0, 0] [108, 45, 11, 347, 95, 330] per iter 0.000214153679323868
285000 [93, 139, 211, 39, 48, 63] [386, 354, 161, 252, 254, 122] per iter 0.0002681096637458132
good [0, 0, 0, 192, 7, 211] [0, 0, 0, 108, 245, 26] : 5.958097053896245e-05 -14.034788850004388
286000 [160, 68, 57, 20, 240, 220] [1, 308, 76, 347, 161, 88] per iter 0.000320970190988554
287000 [244, 79, 243, 15, 141, 255] [226, 69, 408, 331, 218, 51] per iter 0.00037383841222171584
288000 [114, 159, 149, 173, 29, 214] [45, 261, 25

KeyboardInterrupt: 

In [50]:
math.log(tries, 2), math.log(wins, 2), math.log(wins/tries, 2)

(18.09364967270655, 3.9068905956085187, -14.186759077098031)

In [53]:
data = sets, LIN, wins, tries
save(data, "cico_submission_data")